In [16]:

from comet_ml import Experiment
import GPUtil
import torch

import os
import codecarbon
from iesta.machine_learning.dataloader import IESTAData, LABELS
from iesta.machine_learning.huggingface_loader import IESTAHuggingFace
from nlpaf.transformers.text_classification import TextClassification
from dotenv import load_dotenv, find_dotenv
import argparse
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict


In [6]:
data_object = IESTAData(ideology="liberal", keep_labels = LABELS.EFF_INEFF)
huggingface_dataset = IESTAHuggingFace(data_object)

dataset_name = huggingface_dataset.get_dataset_name(is_for_style_classifier=True)
dataset_name

'notaphoenix/debateorg_w_effect_for_liberal_subset'

Found cached dataset parquet (/home/elba_ro/.cache/huggingface/datasets/notaphoenix___parquet/notaphoenix--debateorg_w_effect_for_liberal_subset-2510e01b5ef739a4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [26]:
"adsaA".lower()

'adsaa'

In [24]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

def profile_datasets(ds_name:str = "debateorg_w_effect_for_liberal"):
    dataset_dict: DatasetDict = load_dataset(f"notaphoenix/{ds_name}")
    dfs = []
    for split, ds in dataset_dict.items():
        print(split)
        dfs.append(ds.to_pandas())

    df = pd.concat(dfs)
    len(df)
        #profile = ProfileReport(df, title="Profiling Report")
    df["text_low"] = df["text"].lower()
    profile = ProfileReport(df[["text_low"]], title="Profiling Report")
    profile.to_file(f"../data/profilers/{ds_name}_low.html")

In [25]:
dataset_names = ["debateorg_w_effect_for_liberal_subset",
                 "debateorg_w_effect_for_conservative_subset",
                 "debateorg_w_effect_for_liberal",
                 "debateorg_w_effect_for_conservative"]

for ds_name in dataset_names:
    profile_datasets(ds_name)

Found cached dataset parquet (/home/elba_ro/.cache/huggingface/datasets/notaphoenix___parquet/notaphoenix--debateorg_w_effect_for_liberal_subset-2510e01b5ef739a4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

validation
test
training


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

...

validation
test
training


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]